In [ ]:
import numpy as np
import pandas as pd

class Scaler:
    def __init__(self):
        self.global_min = None
        self.global_max = None
    
    def scale_list_values(self, values):
        return [(x - self.global_min) / (self.global_max - self.global_min) for x in values]

    def scale_1d_array(self, array):
        return (array - self.global_min) / (self.global_max - self.global_min)
    
    def scale(self, x_data, global_min=None, global_max=None):
        # Determine if the input is a DataFrame or a 1D array
        if isinstance(x_data, pd.DataFrame):
            return self._scale_dataframe(x_data, global_min, global_max)
        elif isinstance(x_data, np.ndarray) and x_data.ndim == 1:
            return self._scale_1d_array(x_data, global_min, global_max)
        else:
            raise ValueError("Input should be a DataFrame with a column of lists or a 1D NumPy array.")
    
    def _scale_dataframe(self, df, global_min=None, global_max=None):
        x_data = df.copy()

        # If global_min and global_max are not given, calculate them
        if global_min is None or global_max is None:
            flattened_values = np.concatenate(x_data['Id'].to_list())
            self.global_min = flattened_values.min()
            self.global_max = flattened_values.max()
            print('Found global min:', self.global_min)
            print('Found global max:', self.global_max)
        else:
            # Use provided global_min and global_max
            self.global_min = global_min
            self.global_max = global_max
            print('Using provided global min:', self.global_min)
            print('Using provided global max:', self.global_max)

        # Apply scaling function to each entry in 'Id'
        x_data['Scaled_Id'] = x_data['Id'].apply(lambda values: self.scale_list_values(values))

        return x_data[['Scaled_Id']], self.global_min, self.global_max

    def _scale_1d_array(self, array, global_min=None, global_max=None):
        if global_min is None or global_max is None:
            self.global_min = array.min()
            self.global_max = array.max()
            print('Found global min:', self.global_min)
            print('Found global max:', self.global_max)
        else:
            # Use provided global_min and global_max
            self.global_min = global_min
            self.global_max = global_max
            print('Using provided global min:', self.global_min)
            print('Using provided global max:', self.global_max)

        # Scale the array
        scaled_array = self.scale_1d_array(array)
        return scaled_array, self.global_min, self.global_max
